# Load Dataset 

In [ ]:
from datasets import load_dataset
import polars as pl
dataset = load_dataset("hugosousa/natural_questions_parsed", split="train", streaming=True)

            
            # Convert the dictionary to a Polars DataFrame
# Retrieve and print the first row
# Load the first 1000 rows
first_5000_rows = []
short_answer_count = 0

for i, row in enumerate(dataset):
    if short_answer_count >= 5000:
        break
    if 'short_answers' in row and any(len(inner_list) > 0 for inner_list in row['short_answers']):
        short_answer_count += 1

    first_5000_rows.append(row)

# Convert the list of dictionaries to a Polars DataFrame
df = pl.DataFrame(first_5000_rows)

# Save the DataFrame to a Parquet file
df.write_parquet("/home/student/dynamic-rechunking-RAG/data/first_5000_rows.parquet")

In [ ]:
from datasets import load_dataset

# Load the dataset in streaming mode
dataset = load_dataset('hugosousa/natural_questions_parsed', split='train', streaming=True)

# Initialize a counter

# Iterate over the dataset
for example in dataset:
    # Check if 'short_answers' contains at least one non-empty list with strings
    if 'short_answers' in example and any(len(inner_list) > 0 for inner_list in example['short_answers']):
        short_answer_count += 1
        # Print the short answer
        print(example['short_answers'])
        
        # Stop at 1000
        if short_answer_count >= 1000:
            break


# Creating passages from the documents

In [4]:
import spacy.cli
import polars as pl
nlp = spacy.load("en_core_web_sm")
df=pl.read_parquet("/home/tomer/learning/dynamic-rechunking-RAG/data/short_answer_5000_rows.parquet")
def divide_with_spacy(text, target_length=200):
    """
    Divide text into passages using spaCy for sentence segmentation.

    :param text: The input document as a string.
    :param target_length: Approximate number of words per passage.
    :return: A list of passages.
    """
    doc = nlp(text)
    
    # Extract sentences
    sentences = [sent.text for sent in doc.sents]
    
    # Group sentences into passages
    passages = []
    current_passage = []
    current_length = 0
    
    for sentence in sentences:
        words = sentence.split()
        sentence_length = len(words)
        
        if current_length + sentence_length <= target_length:
            current_passage.append(sentence)
            current_length += sentence_length
        else:
            passages.append(" ".join(current_passage))
            current_passage = [sentence]
            current_length = sentence_length

    # Add the last passage
    if current_passage:
        passages.append(" ".join(current_passage))

    return passages
print(df.columns)

df = df.with_columns(
    pl.col("document").map_elements(lambda doc: divide_with_spacy(doc), return_dtype=pl.List(pl.Utf8)).alias("document_passages")
)

['question', 'long_answer', 'short_answers', 'yes_no_answer', 'candidates', 'document', 'id']


In [7]:
# Initialize an empty dictionary
import json
df.write_parquet("/home/tomer/learning/dynamic-rechunking-RAG/data/short_answer_5000_rows.parquetshort_answer_5000_rows_with_passages.parquet")
passage_to_location = {}

# Iterate over the DataFrame rows
for row_idx, row in enumerate(df.iter_rows(named=True)):
    passages = row["document_passages"]  # Extract the list of passages
    for passage_idx, passage in enumerate(passages):
        # Map each passage to its document row and index
        passage_to_location[f"{row_idx}_{passage_idx}"] = {"passage": passage, "document_row": row_idx, "passage_index": passage_idx}

# Print the resulting dictionary
json.dump(passage_to_location, open("/home/tomer/learning/dynamic-rechunking-RAG/data/5000_passage_to_location.json", "w"), indent=4)

# Create answer data as json


In [ ]:
import polars as pl
df_loaded = pl.read_parquet("/home/tomer/dynamic-rechunking-RAG/data/first_1000_rows_with_passeges.parquet")

# Initialize an empty list to store the extracted data
extracted_data = []

# Iterate over the DataFrame rows
for row_idx, row in enumerate(df_loaded.iter_rows(named=True)):
    extracted_data.append({
        "row_number": row_idx,
        "question": row["question"],
        "long_answer": row["long_answer"],
        "short_answer": row["short_answers"],
        "yes_no_answer": row["yes_no_answer"],
        "id": row["id"]
    })

# Save the extracted data to a JSON file
with open("/home/tomer/dynamic-rechunking-RAG/data/answer_json.json", "w") as json_file:
    json.dump(extracted_data, json_file, indent=4)

# Create embdedings and upserting to Vector DB

In [2]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm
import json
# Initialize the model
model = SentenceTransformer('intfloat/e5-base-v2')

# Initialize Pinecone
pinecone_index_name="dynamic"
pinecone_api_key="pcsk_5SLo1E_SPpKBieD8wQRuRf9G7xfYrVWDLWUAkfRC3X5xjMpDhy7j8CH3SEen8kJiNbMjav"
pc=Pinecone(api_key=pinecone_api_key)

# Create Pinecone index if it doesn't exist
print("Creating a Pinecone index...")
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if pinecone_index_name not in existing_indexes:
    pc.create_index(
        name=pinecone_index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
pinecone_index=pc.Index(pinecone_index_name)
with open("/home/tomer/dynamic-rechunking-RAG/data/passage_to_location.json", "r") as file:
    data = json.load(file)
# index=pc.Index(pinecone_index_name)
# to_upsert=[]
# batch_size = 64
# print("create embeddings...")
# for key, value in tqdm(data.items()):
#     passage = value.get("passage", "")
#     if key=="0_0":
#         print(passage)
#     document_row = value.get("document_row", None)
#     passage_index = value.get("passage_index", None) 
#     if passage:
#         embedding = model.encode(passage)
#     else:
#         embedding = None
#         continue        # Check if the "passages" structure is valid

#         # Prepare upsert data for the current row
#     to_upsert.append({
#         "id": str(key),
#         "values": embedding.tolist(),
#         "metadata": {"row_number":document_row, "location": passage_index, "passage":passage}       
#     })
            
#         # Upsert data to Pinecone
        
       
# # Upsert in batches using tqdm for progress tracking
# print("Upserting the embeddings to the Pinecone index...")
# for i in tqdm(range(0, len(to_upsert), batch_size)):
#     i_end = min(i + batch_size, len(to_upsert))
#     batch = to_upsert[i:i_end]
#     pinecone_index.upsert(vectors=batch)
# print("Dense indexing completed.")





/home/tomer/dynamic-rechunking-RAG/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating a Pinecone index...


In [3]:



def search_dense(query, top_k=3):
    """
    Perform a dense search using Pinecone.

    :param query: The search query as a string.
    :param top_k: Number of top results to retrieve.
    :return: List of (doc_id, score, metadata) tuples.
    """
    # Generate the embedding for the query question
    query_embedding = model.encode(query, convert_to_numpy=True)

    # query_embedding = embed_text(query)

    # Perform the query with Pinecone
    try:
        response =pinecone_index.query(
            vector=query_embedding.tolist(),
            top_k=top_k,
            include_metadata=True
        )
        # Extract the matches with metadata
        matches = response['matches']
        hits = [(match['id'], match['score']) for match in matches]
    except Exception as e:
        print(f"Error querying index: {e}")
        hits = []

    return hits


initial_retrieval=search_dense("what new deal acts are still in effect today", top_k=10)
print(initial_retrieval)

[('0_2', 0.865054905), ('0_152', 0.853706121), ('313_78', 0.853706121), ('0_6', 0.850366712), ('313_4', 0.845571578), ('0_1', 0.844175), ('440_12', 0.844043612), ('0_148', 0.844043612), ('374_18', 0.844043612), ('0_139', 0.842032075)]


In [6]:


###################### threshold logic
first_value = initial_retrieval[0][1]
last_value = initial_retrieval[-1][1]
# Calculate the difference
interval_thereshold = first_value-last_value

from  expansion_service import ExpansionService
query="what new deal acts are still in effect today"

new_passages=[]
es = ExpansionService()
for i,entry in enumerate(initial_retrieval):
    # print("before:         ",data[entry[0]]["passage"])
    expanded_passage=es.expand_passages(entry[0])
    distilled_passage=es.query_focused_rechunk(query, expanded_passage,float(entry[1])-interval_thereshold)
    new_passages.append(distilled_passage)
    print(f" passage {i}: after           ",distilled_passage)


print("reranking")
rerenaked_passages=es.rerank_chunks( query,new_passages)





 passage 0: after            New Deal Top left: The Tennessee Valley Authority , part of the New Deal, being signed into law in 1933 Top right: President Roosevelt was responsible for initiatives and programs of the New Deal Bottom: A public mural from one of the artists employed by the Works Progress Administration , part of the New Deal Duration 1933-- 1937 Location United States Type Economic program Cause Great Depression Organized by President Franklin D. Roosevelt Secretary Henry Morgenthau Jr. Secretary Harry Hopkins Secretary Frances Perkins Participants Society of the United States Outcome Disputed / American economic recovery/Lasting unemployment relief Part of a series on Progressivism History (show) Atomic Age Enlightenment Industrial Age Information Age Jet Age Machine Age Progressive Era Space Age Ideas (show) Economic development Broad measures Economic growth Empirical evidence Direct democracy Freedom of movement Human enhancement Idea of Progress Industrialisation Lin

In [10]:
rerenaked_passages[1][0]

"Sara Delano Roosevelt (granddaughter) Franklin Delano Roosevelt III (grandson) John Roosevelt Boettiger (grandson) James Roosevelt III (grandson) James Roosevelt I (father) Sara Ann Delano (mother) James Roosevelt Roosevelt (half-brother) Isaac Roosevelt (grandfather) Jacobus Roosevelt (great-grandfather) Fala (family dog) ←\nHerbert Hoover Harry S. Truman → Category ( hide ) New Deal Causes and legacy Great Depression New Deal Coalition Brain Trust American Liberty League Criticism New Deal Emergency Banking Act Economy Act Agricultural Adjustment Act Civilian Conservation Corps (CCC) Civil Works Administration Communications Act Executive Order 6102 Homeowners Refinancing Act Farm Credit Administration Federal Deposit Insurance Corporation (FDIC) Federal Emergency Relief Administration Frazier-- Lemke Farm Bankruptcy Act Glass-- Steagall Act National Industrial Recovery Act National Housing Act National Recovery Administration National Youth Administration Public Works Administratio

In [11]:
from ollama_interface import OllamaInterface
def create_rag_prompt(question, retrieved_context, k=2):
    """
    Create a RAG prompt dynamically using the top-k retrieved passages.

    :param question: The question to be answered.
    :param retrieved_context: A list of tuples where each tuple contains a passage and its relevance score.
    :param k: Number of passages to include in the prompt.
    :return: A formatted string containing the RAG prompt.
    """
    passages = "\n".join([f"passage {i + 1}: {retrieved_context[i][0]}" for i in range(min(k, len(retrieved_context)))])
    
    return f"""
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>

            Answer the question based on the given passages below. Keep the answer concise.
            <|eot_id|><|start_header_id|>user<|end_header_id|>

            ## Passages:
            {passages}

            ## Question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>

            ##Answer:
            """

OI=OllamaInterface()
rag_prompt=create_rag_prompt(query, rerenaked_passages)
OI.send_message(rag_prompt)



'Several New Deal acts are still in effect today. Some examples include:\n\n1. Social Security Act (1935)\n2. Federal Deposit Insurance Corporation (FDIC) created by the Glass-Steagall Act (1933)\n3. Tennessee Valley Authority (TVA) is still operational\n4. Securities and Exchange Commission (SEC) created by the Securities Act (1933)\n5. National Labor Relations Board (NLRB) established by the Wagner Act (1935)\n\nThese programs have been modified or updated over time, but they continue to play a significant role in the US economy today.'

In [13]:
import pandas as pd
def get_queries(path):
    df = pd.read_parquet(path)
    questions = df['question'].head(1000).tolist()
    return questions
get_queries("/home/tomer/dynamic-rechunking-RAG/data/first_1000_rows.parquet")

['what new deal acts are still in effect today',
 'major blood vessels that carry blood away from the heart',
 'when did st louis win the super bowl',
 'where did the term wise guy come from',
 'seats reserved for sc and st in loksabha',
 'where did the council of nicea take place',
 'where was the empire hall and what was it originally built as in 1904',
 'who won american idol the year jennifer hudson',
 "what a long strange trip it's been grateful dead",
 'who plays timmy dad in a fairly odd movie',
 'what is the difference between duoderm and exuderm',
 'which technique was mostly used by the artists of bengal school',
 'which river is not flowing into bay of bengal',
 'who won the mayoral race in st petersburg',
 'unix portmapper remote infomation retrieving attempt(32796)',
 'who is the best coach for indian cricket team',
 'when would the priest enter the holy of holies',
 'who said with great power comes great responsibility quote',
 'the first law of thermodynamics is a re – s

In [ ]:
pl